## Test Tool Calling from AI Agent

### Setting-up dotEnv & LLM 

In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv(override=True)
load_dotenv('./../.env')

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.5,
    max_tokens=300
)

### Connect to DeepEval

In [2]:
import deepeval

deepeval.login_with_confident_api_key("confident_us_DNEaOTujz584SicCTCNGpP7snhHa3APERq7FhaTvpDI=")

🎉🥳 Congratulations! You've successfully logged in! 🙌

### Creating the Tools for AI Agent

In [3]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    "Use this tool to Add two numbers and return the result"
    return a+b

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Use this tool to Subtract two numbers and return the result"
    return a-b

@tool
def multiply_numbers(a: float, b: float) -> float:
    "Use this tool to Multiply two numbers and return the result"
    return a*b


In [4]:
list_of_tools = [add_numbers, subtract_numbers, multiply_numbers]

### Setting-up AI Agent to Invoke Custom Tooling

In [5]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    llm=llm,
    tools=list_of_tools,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True
)

response = agent.invoke("I had 35 dollars. I spent 20 & got a 10 percent cashback, how much do I have left?")

print(response['intermediate_steps'])

/var/folders/xl/2h_vnm9s4dl0fnn4x8_mg4q40000gn/T/ipykernel_40942/3494069479.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
<think>
Okay, let's see. The user started with $35. They spent $20, so first I need to subtract 20 from 35. That would leave them with $15. But then they got a 10% cashback. Wait, cashback on what? Probably on the $20 spent. So 10% of 20 is $2. So they get $2 back. Adding that to the remaining $15 would give $17. Let me check the steps again. Subtract 20 from 35, then add 10% of 20. So 35 - 20 = 15, then 15 + 2 = 17. Yeah, that makes sense. So the final amount is $17.
</think>

Thought: I need to calculate the remaining money after spending and adding the cashback. First, subtract the spent amount from the initial amount, then add the cashback. 

Action:
```
{
  "action": "subtract_numbers",
  "action_input": {
    "a": 35,
    "b": 20
  }
}
```


Observation: 15
Thought:<think>
</think>

Action:
```
{
  "action": "multiply_numbers",
  "action_input": {
    "a": 20,
    "b": 0.1
  }
}
```



Observation: 2.0
Thought:<think>
Okay, let me figure th

### Preparing Golden Dataset for DeepEval Testcase

In [ ]:
from deepeval.test_case import ToolCall

golden_dataset = [

    {
        "question": "What is 10 - 4?",
        "expected_answer": "The difference between 10 and 4 is 6.",
        "tool_called": [
            ToolCall(name="subtract_numbers", input_parameters={"a": 10, "b": 4})
        ]
    },
    {
        "question": "What is 2.5 * 4?",
        "expected_answer": "The product of 2.5 and 4 is 10.0.",
        "tool_called": [
            ToolCall(name="multiply_numbers", input_parameters={"a": 2.5, "b": 4})
        ]
    },
    {
        "question": "My salary was $9500 & I got a 15 percent raise yesterday, how much do I earn now?",
        "expected_answer": "You now earn $10925 after a 15 percent raise on your previous salary of $9500."  ,
        "tool_called": [
            ToolCall(name="multiply_numbers", input_parameters={"a": 9500, "b": 1.15})
        ]
    },
    {
        "question": "I had 35 dollars. I spent 20 & got a 10 percent cashback, how much do I have left?",
        "expected_answer": "You have a total of 17 dollars left after spending 20 dollars and receiving a 10 percent cashback of 2 dollars.",
        "tool_called": [
            ToolCall(name="subtract_numbers", input_parameters={"a": 35, "b": 20}),
            ToolCall(name="multiply_numbers", input_parameters={"a": 20, "b": 0.1}),
            ToolCall(name="add_numbers", input_parameters={"a": 15, "b": 2})
        ]
    }
]

golden_dataset

[{'question': 'What is 10 - 4?',
  'expected_answer': 'The difference between 10 and 4 is 6.',
  'tool_called': [ToolCall(
       name="subtract_numbers",
       input_parameters={
           "a": 10,
           "b": 4
       }
   )]},
 {'question': 'What is 2.5 * 4?',
  'expected_answer': 'The product of 2.5 and 4 is 10.0.',
  'tool_called': [ToolCall(
       name="multiply_numbers",
       input_parameters={
           "a": 2.5,
           "b": 4
       }
   )]},
 {'question': 'My salary was $9500 & I got a 15 percent raise yesterday, how much do I earn now?',
  'expected_answer': 'You now earn $10925 after a 15 percent raise on your previous salary of $9500.',
  'tool_called': [ToolCall(
       name="multiply_numbers",
       input_parameters={
           "a": 9500,
           "b": 1.15
       }
   )]},
 {'question': 'I had 35 dollars. I spent 20 & got a 10 percent cashback, how much do I have left?',
  'expected_answer': 'You have a total of 17 dollars left after spending 20 dollar

### Upload Golden Dataset to DeepEval

In [7]:
from deepeval.dataset import EvaluationDataset, Golden

goldens = []

for data in golden_dataset:
    golden = Golden(
        input=data['question'],
        expected_output=data['expected_answer'],
        expected_tools=data['tool_called']
    )
    goldens.append(golden)
    
eval_dataset = EvaluationDataset(goldens=goldens)

eval_dataset

EvaluationDataset(test_cases=[], goldens=[Golden(input='What is 10 - 4?', actual_output=None, expected_output='The difference between 10 and 4 is 6.', context=None, retrieval_context=None, additional_metadata=None, comments=None, tools_called=None, expected_tools=[ToolCall(
    name="subtract_numbers",
    input_parameters={
        "a": 10,
        "b": 4
    }
)], source_file=None, name=None, custom_column_key_values=None), Golden(input='What is 2.5 * 4?', actual_output=None, expected_output='The product of 2.5 and 4 is 10.0.', context=None, retrieval_context=None, additional_metadata=None, comments=None, tools_called=None, expected_tools=[ToolCall(
    name="multiply_numbers",
    input_parameters={
        "a": 2.5,
        "b": 4
    }
)], source_file=None, name=None, custom_column_key_values=None), Golden(input='My salary was $9500 & I got a 15 percent raise yesterday, how much do I earn now?', actual_output=None, expected_output='You now earn $10925 after a 15 percent raise on y

In [8]:
eval_dataset.push("TestToolCalling Dataset")

In [9]:
import re

# Remove <think> blocks from the LLM response
def remove_think_block(text) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL | re.IGNORECASE)

### Creating LLMTestCase and Invoking the AI Agent

In [10]:
def query_agent(question):
    
    agent_response = agent.invoke(question)
    intermidiate_steps = agent_response['intermediate_steps']

    tools_called_list=[]
    
    for agent_action, result in intermidiate_steps:    
        tools_called_list.append(ToolCall(name=agent_action.tool, input_parameters=agent_action.tool_input, output=result))
    
    final_response = remove_think_block(agent_response['output'])
    
    return tools_called_list, final_response
    
# response = query_agent("I had 35 dollars. I spent 20 & got a 10 percent cashback, how much do I have left?")    
# response

In [11]:
from deepeval.test_case import LLMTestCase

for golden in goldens:
    input = golden.input
    tools_called_list, response = query_agent(input)
 
    test_case = LLMTestCase(
        input=input,
        actual_output=response,
        expected_tools=golden.expected_tools,
        expected_output=golden.expected_output,
        tools_called=tools_called_list
    )

    eval_dataset.add_test_case(test_case)

eval_dataset.test_cases



> Entering new AgentExecutor chain...
<think>
Okay, the user is asking "What is 10 - 4?" So I need to subtract 4 from 10. Let me check which tool to use. The available tools are add_numbers, subtract_numbers, and multiply_numbers. The subtract_numbers tool is the right one here.

The parameters for subtract_numbers are a and b, both integers. The user's question is 10 minus 4, so a should be 10 and b should be 4. I'll call the subtract_numbers function with these values. The result should be 6. Then I'll present that as the final answer.
</think>

Thought: I need to subtract 4 from 10 using the subtract_numbers tool.
Action:
```
{
  "action": "subtract_numbers",
  "action_input": {
    "a": 10,
    "b": 4
  }
}
```

Observation: 6
Thought:<think>
</think>

Action:
```
{
  "action": "Final Answer",
  "action_input": "The result of 10 - 4 is 6."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
<think>
Okay, let's see. The user asked for 2.5 multiplied by 4. I need to us

[LLMTestCase(input='What is 10 - 4?', actual_output='The result of 10 - 4 is 6.', expected_output='The difference between 10 and 4 is 6.', context=None, retrieval_context=None, additional_metadata=None, tools_called=[ToolCall(
     name="subtract_numbers",
     input_parameters={
         "a": 10,
         "b": 4
     },
     output=6
 )], comments=None, expected_tools=[ToolCall(
     name="subtract_numbers",
     input_parameters={
         "a": 10,
         "b": 4
     }
 )], token_cost=None, completion_time=None, name=None, tags=None),
 LLMTestCase(input='What is 2.5 * 4?', actual_output='The result of 2.5 * 4 is 10.0.', expected_output='The product of 2.5 and 4 is 10.0.', context=None, retrieval_context=None, additional_metadata=None, tools_called=[ToolCall(
     name="multiply_numbers",
     input_parameters={
         "a": 2.5,
         "b": 4
     },
     output=10.0
 )], comments=None, expected_tools=[ToolCall(
     name="multiply_numbers",
     input_parameters={
         "a":

### Evaluating ToolCorrectnessMetric

In [22]:
from deepeval.metrics import ToolCorrectnessMetric

for test_case in eval_dataset:
    metric = ToolCorrectnessMetric(should_consider_ordering=True)
    metric.measure(test_case)
    
    print(test_case.input)
    print(metric.score)
    print(metric.reason)
    print(test_case.expected_tools)

Output()

Output()

What is 10 - 4?
1.0
Correct ordering: all expected tools ['subtract_numbers'] were called in the correct order.
[ToolCall(
    name="subtract_numbers",
    input_parameters={
        "a": 10,
        "b": 4
    }
)]


Output()

What is 2.5 * 4?
1.0
Correct ordering: all expected tools ['multiply_numbers'] were called in the correct order.
[ToolCall(
    name="multiply_numbers",
    input_parameters={
        "a": 2.5,
        "b": 4
    }
)]


Output()

My salary was $9500 & I got a 15 percent raise yesterday, how much do I earn now?
1.0
Correct ordering: all expected tools ['multiply_numbers'] were called in the correct order.
[ToolCall(
    name="multiply_numbers",
    input_parameters={
        "a": 9500,
        "b": 1.15
    }
)]


I had 35 dollars. I spent 20 & got a 10 percent cashback, how much do I have left?
1.0
Correct ordering: all expected tools ['subtract_numbers', 'multiply_numbers', 'add_numbers'] were called in the correct order.
[ToolCall(
    name="subtract_numbers",
    input_parameters={
        "a": 35,
        "b": 20
    }
), ToolCall(
    name="multiply_numbers",
    input_parameters={
        "a": 20,
        "b": 0.1
    }
), ToolCall(
    name="add_numbers",
    input_parameters={
        "a": 15,
        "b": 2
    }
)]
